In [ ]:
import pandas as pd

In [ ]:
#examine current cross_walk file
#calm = pd.read_csv("data/geo_cross_walk.csv")
#calm.head(2)

In [ ]:
%cd ..

# read and prepare block file

In [ ]:
sem = pd.read_csv("data/semcog_block2010.txt")



In [ ]:
#read semcog block GIS file
sem = pd.read_csv("data/semcog_block2010.txt") 
semblk = sem.loc[sem.SUBTYPECD == 1][['GEOID10']]
print(semblk.dtypes)
semblk['GEOID10'].head(2)

In [ ]:
semblk['BLKGRPID'] = semblk['GEOID10'].apply(str).str[:12].apply(int) # get block group id
semblk.drop_duplicates(subset = ['BLKGRPID'], inplace = True)  # drop duplicated BG
semblk['TRACT'] = semblk['GEOID10'].apply(str).str[5:11].apply(int)
semblk['BLKGRP'] = semblk['GEOID10'].apply(str).str[11].apply(int)
semblk['TRACTID'] = semblk['GEOID10'].apply(str).str[:11].apply(int)

In [ ]:
# read 2010 tract to puma file from Census
tract_puma = pd.read_csv("data/2010_Census_Tract_to_2010_PUMA.txt")
tract_puma = tract_puma.loc[tract_puma.STATEFP == 26]
tract_puma['TRACTID'] = tract_puma['STATEFP'] * 1000000000 + tract_puma['COUNTYFP'] * 1000000 + tract_puma['TRACTCE']
tract_puma.head(2)

In [ ]:
tract_puma.loc[tract_puma.duplicated(subset=['TRACTID'])]
#semblk.loc[semblk.duplicated(subset=['TRACTID'])]

In [ ]:
# merge block group file with puma file
semblkgrp = pd.merge(semblk, tract_puma, left_on = 'TRACTID', right_on = 'TRACTID', how = 'left')


In [ ]:
semblkgrp.dtypes

In [ ]:
semblkgrp['REGION'] = 4
semblkgrp.rename(columns = {'PUMA5CE': 'PUMA'}, inplace = True)

In [ ]:
semblkgrp.to_csv("data/sem_geo_cross_walk.csv")

In [ ]:
oakblkgrp = semblkgrp.loc[semblkgrp.COUNTYFP == 125 ]
oakblkgrp.to_csv("data/oak_geo_cross_walk.csv")

In [ ]:
# add PUMA to controls
dfc = pd.read_csv("data/oak_control_totals_blkgrp.csv")
dfc['blkgrpid'] = dfc['STATEFP'] * 10000000000 + dfc['COUNTYFP'] * 10000000 + dfc['TRACTCE'] * 10 + dfc['BLKGRPCE']

dfg = pd.read_csv("data/sem_geo_cross_walk.csv")
print(dfc.columns.values)
print(dfg.columns.values) 

In [ ]:
dfc = pd.merge(dfc, dfg[['BLKGRPID', 'PUMA']], left_on='blkgrpid', right_on='BLKGRPID', how='left')

In [ ]:
dfc.to_csv("data/oak_control_totals_blkgrp.csv")